<a href="https://colab.research.google.com/github/klnsuman/MachineLearning/blob/master/SVM_Anamoly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [25]:
import numpy as np
import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

sns.set(style='whitegrid', palette='muted')
rcParams['figure.figsize'] = 14, 8
np.random.seed(1)
tf.random.set_seed(1)

print('Tensorflow version:', tf.__version__)

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version: 2.3.0


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os
os.chdir('/content/drive/My Drive/Anamoly')

In [28]:
!ls -lrt

total 122710
-rw------- 1 root root    274780 Nov 20 04:51 faccount.txt
-rw------- 1 root root 125352890 Nov 20 04:52 ftxn2.txt
-rw------- 1 root root       366 Nov 20 04:53 fagraph.ddl
-rw------- 1 root root     13371 Nov 21 05:58 fruits_hist.png
-rw------- 1 root root     12198 Nov 21 06:03 Amt.png


In [29]:
df_facct = pd.read_csv('faccount.txt')

df_ftxn = pd.read_csv('ftxn2.txt')


df_ftxn["TXNDate"]= pd.to_datetime(df_ftxn["TXDATE"])


In [30]:
from sklearn import cluster
from sklearn import metrics
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [32]:
df_ftxn["TXNhour"] = df_ftxn["TXNDate"].dt.hour
df_ftxn["TXNDay"] = df_ftxn["TXNDate"].dt.day
df_ftxn["TXNMnth"] = df_ftxn["TXNDate"].dt.month
df_ftxn["TXNWDay"] = df_ftxn["TXNDate"].dt.dayofweek

In [35]:
train['AMOUNT']

0        -0.485225
1        -0.570332
2         1.357413
3        -0.564700
4        -0.570645
            ...   
766528    0.146197
766529   -0.482721
766530   -0.567516
766531    1.449092
766532   -0.473960
Name: AMOUNT, Length: 766533, dtype: float64

In [36]:
train_size = int(len(df_ftxn) * 0.5)
test_size = len(df_ftxn) - train_size
train, test = df_ftxn.iloc[0:train_size], df_ftxn.iloc[train_size:len(df_ftxn)]
print(train.shape, test.shape)

(766533, 14) (766533, 14)


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(train[['AMOUNT']])

train['AMOUNT'] = scaler.transform(train[['AMOUNT']])
test['AMOUNT'] = scaler.transform(test[['AMOUNT']])

In [39]:
train.head()

,TXN_ID,TXTYPE,AMOUNT,FROMACCTID,TOACCTID,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,TXNDate,TXNhour,TXNDay,TXNMnth,TXNWDay
0,950060711303,PAYMENT,-0.485225,500607,800242,2020-01-22T17:00:00,Dinner with Shane,N,Y,2020-01-22 17:00:00,17,22,1,2
1,950060711401,PAYMENT,-0.570332,500607,800111,2020-01-23T08:00:00,Breakfast with Steven,N,Y,2020-01-23 08:00:00,8,23,1,3
2,950060711402,TRANSFER,1.357413,500607,500929,2020-01-23T13:00:00,Holiday Cost,N,Y,2020-01-23 13:00:00,13,23,1,3
3,950060711403,PAYMENT,-0.564700,500607,800252,2020-01-23T17:00:00,Meal with Mary,N,Y,2020-01-23 17:00:00,17,23,1,3
4,950060711501,PAYMENT,-0.570645,500607,800116,2020-01-24T08:00:00,Coffee with Brian,N,Y,2020-01-24 08:00:00,8,24,1,4


In [44]:
X = train[["TXNhour","TXNDay","TXNMnth","AMOUNT"]]
#X = X.reshape(-1, 1) 
kmeans_model = cluster.KMeans(n_clusters=2, random_state=1)
kmeans_model.fit(X)
kmeans_model.cluster_centers_
kmeans_model.labels_

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [45]:
a = kmeans_model.labels_
len(a)

766533

In [46]:
result = np.where(a == 1)
result

(array([     0,      1,      2, ..., 766530, 766531, 766532]),)

In [47]:
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
svm_model = svm.OneClassSVM(nu=0.1, gamma=4.0)
svm_model.fit(X)
decisions_svm = svm_model.decision_function(X)